In [316]:
import json
import time
import requests
import numpy as np
from tqdm.auto import tqdm
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [317]:
current_datetime = datetime.now()

In [318]:
filename = f'linii_lubvi_{current_datetime.day}_{current_datetime.month}.xlsx'

In [319]:
cookies = {
    '__ddgid_': 'JX58qbbb3PHdWZ43',
    '__ddg2_': 'SasnCXcXaSPjKfkF',
    '__ddg1_': 'wLWQw8kuYGGbgxttlHm2',
    'experimentVariantId_ll8': '11',
    'subscribe-popup': 'cookiesubscribe',
    'rrpvid': '822158495319251',
    '_ym_uid': '1698041660594023635',
    '_ym_d': '1698041660',
    'tmr_lvid': '53c65497a757cbb85074d9e298748caa',
    'tmr_lvidTS': '1698041659870',
    'rcuid': '6527aace80057fba2add642e',
    'BX_USER_ID': '6d8e8ec7412fc8bf94b4e13d087ffa5f',
    '_userGUID': '0:lo2i5umr:ORWCpvmo~q0hArF1bVbRQxprFHm4mxD2',
    'popmechanic_sbjs_migrations': 'popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1',
    'BITRIX_SM_UIDH': 'ba56e56642279fea4b479ba31d92b4e5',
    'BITRIX_SM_UIDL': 'LL_79093180245',
    'BITRIX_SM_SALE_UID': '0',
    '__utmz': '117815316.1706506202.10.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
    'advcake_track_id': 'ed260e2f-f044-66c5-ec29-60132d81d363',
    'advcake_session_id': '9ea27407-3a65-709f-99e7-01d641ed886b',
    '_gid': 'GA1.2.475253905.1709724082',
    '_ym_isad': '1',
    'BITRIX_SM_SOUND_LOGIN_PLAYED': 'Y',
    '_ga_cid': '378912807.1698041660',
    'BITRIX_CONVERSION_CONTEXT_od': '%7B%22ID%22%3A1%2C%22EXPIRE%22%3A1709845140%2C%22UNIQUE%22%3A%5B%22conversion_visit_day%22%5D%7D',
    '__utma': '117815316.378912807.1698041660.1709724083.1709793716.16',
    '__utmc': '117815316',
    '__utmt_UA-32426418-13': '1',
    'dSesn': '927bd3c3-6e52-4584-f940-cb41aaf2414e',
    '_dvs': '0:ltgv1742:55CGZykIuGpd4ewHt~U8PMQbUD3aUZ2L',
    'digi_uc': 'W1sidiIsIjEwMDE5OTY5IiwxNzA5NzI1MzMwNzU0XSxbInYiLCI4NDQ5OTQ2IiwxNzA5NzkzNzE2NTU3XSxbInYiLCIxMDM1OTQ2MyIsMTcwOTI4MzA0OTUyMV0sWyJ2IiwiMTAzNTkzNzUiLDE3MDkyODMwNDI5MjNdLFsidiIsIjEwMzU5MzczIiwxNzA5MjgzMDM0ODQzXV0=',
    'full': '0',
    'PHPSESSID': '1qop03lqnadl6slqja15k3bbsf',
    '_gat': '1',
    '_ga_Q2GKTC7MRN': 'GS1.1.1709793716.18.1.1709793812.0.0.0',
    '_ga': 'GA1.1.378912807.1698041660',
    '__utmb': '117815316.9.10.1709793716',
    '_ga_P3KP5WRTY8': 'GS1.2.1709793716.15.1.1709793812.47.0.0',
    'tmr_detect': '1%7C1709793812253',
    'mindboxDeviceUUID': '6e9424e5-a80e-43b6-b480-b1d86e4084fd',
    'directCrm-session': '%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
}
headers = {'user-agent': UserAgent().random}

In [320]:
response = requests.get(f'https://liniilubvi.ru/catalog/zolotye/proba_585/', headers=headers, cookies=cookies)

In [324]:
soup = bs(response.content, 'html.parser')
pages = soup.find_all('a', class_='pager-item')[-1].get('href')
id = pages.find('=')
total = int(pages[id+1:])

paginator = pages.find('?')
paginator = pages[paginator:pages.find('=')+1]

In [325]:
urls = list()
prices = list()

In [326]:
for page in tqdm(range(1, total + 1)):
    response = requests.get(f'https://liniilubvi.ru/catalog/zolotye/proba_585/{paginator}{page}', headers=headers, cookies=cookies)
    soup = bs(response.content, 'html.parser')
    links = soup.find_all('div', class_='b-fade-slider mob-hide-1024')
    prices_ = soup.find_all('span', class_='carusel-wrap-line-center-item-list-item-price price_rub')
    if len(prices_) == len(links):
        for link in links:
            urls.append(link.get('href'))
        for price in prices_:
            prices.append(price.text.strip())

  0%|          | 0/132 [00:00<?, ?it/s]

In [327]:
df = pd.DataFrame({'links': urls, 'price': prices}).drop_duplicates(ignore_index=True)

In [328]:
groups = list()
names = list()
weights = list()
params = list()

In [329]:
for link in tqdm(df['links'].tolist()):
    try:
        response = requests.get(f'https://liniilubvi.ru{link}', headers=headers, cookies=cookies).content
    except Exception as ex:
        print(ex)
        groups.append(np.nan)
        names.append(np.nan)
        weights.append(np.nan)
        prices.append(np.nan)
        params.append(np.nan)
        continue
    
    soup = bs(response, 'html.parser')
    
    try:
        group = soup.find('span', class_="breadcrumbs__item-text").text
    except:
        group = np.nan
        
    try:
        price = soup.find('a', class_="btn red btn--block").text.strip()
        try:
            price = soup.find('div', class_="pro-price-block").find('span', class_="pro-price-item").text
        except:
            price = np.nan
    except:
        price = np.nan
        
    try:
        name = soup.find('div', class_="title-border title-border--mb").find('h1', {'itemprop': 'name'}).text
    except:
        name = np.nan

    try:
        weight = soup.find('div', class_="param-item-right middle-weight").text
    except:
        weight = np.nan
    
    # params_names = []
    # params_values = []
    # param = soup.find_all('li', class_='param-item clearfix')
    # len_params = len(param)
    # for i in range(len_params):
    #     try:
    #         name = params[i].find('div', class_='param-item-left').text
    #         params_names.append(name)
    #     except:
    #         params_names.append(np.nan)
    #     try:
    #         value = params[i].find('div', class_='param-item-right 111').text
    #         params_values.append(value)
    #     except:
    #         params_values.append(np.nan)

    # linii_params = dict(zip(params_names, params_values))
    
    groups.append(group)
    names.append(name)
    weights.append(weight)
    prices.append(price)
    # params.append(linii_params)    

  0%|          | 0/1733 [00:00<?, ?it/s]

In [330]:
cards = pd.DataFrame({'name': names, 'weight': weights, 'group': groups})

In [332]:
df = pd.concat([df, cards], axis=True)

In [352]:
def fill_weight(weight):
    try:
        int(weight[0])
        return float(weight)
    except:
        return -1

In [ ]:
df['weight'] = df['weight'].str.replace(' г', '')
df['weight'] = df['weight'].apply(fill_weight)
df['price'] = df['price'].str.replace(' ', '').astype('int')

In [356]:
df_test = df[df['weight'] > -1]

In [362]:
def fill_mean(weight):
    if weight < 1:
        return df_test['weight'].mean()
    return weight

In [363]:
df['weight'] = df['weight'].apply(fill_mean)

In [364]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   links   1733 non-null   object 
 1   price   1733 non-null   int32  
 2   name    1733 non-null   object 
 3   weight  1733 non-null   float64
 4   group   1733 non-null   object 
 5   params  1733 non-null   object 
dtypes: float64(1), int32(1), object(4)
memory usage: 74.6+ KB


In [365]:
df.to_excel(f'C:\\Users\Trenkin.Sergey\Desktop\parsers\workspace\{filename}')